<a href="https://colab.research.google.com/github/jsblandon/era5_data/blob/main/era5_data_download_management_gh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Conexión por API al servidor CDS**

La conexión al servidor se basa en el tutorial expuesto en el siguiente [foro](https://stackoverflow.com/questions/64304862/using-cdsapi-in-google-colab).

In [ ]:
# Se crea el archivo para agregar el Usuario y la clave de la API de la página
# Ir a https://cds.climate.copernicus.eu/profile

url = "https://cds.climate.copernicus.eu/api"
apikey = 'aqui-va-la-api-de-tu-perfil'

# **Instalación de paquetes**

In [ ]:
%%capture

# Climate Data Store API
!pip install cdsapi

# Plot data on map projections with matplotlib
!pip install basemap

# Provides an object-oriented python interface to the netCDF version 4 library
!pip install netcdf4

# Raster managing package installing
!pip install rioxarray

In [ ]:
ignore_warnings = True                                        # ignore warnings?
import cdsapi
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import netCDF4 as nc
import rioxarray
import xarray as xr
import numpy as np
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator) # control of axes ticks
from matplotlib.colors import ListedColormap                  # custom color maps
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns                                         # advanced plotting
import matplotlib.ticker as mtick                             # control tick label formatting
plt.rc('axes', axisbelow=True)                                # plot all grids below the plot elements
if ignore_warnings == True:
    import warnings
    warnings.filterwarnings('ignore')
cmap = plt.cm.YlOrRd                                         # color map

In [ ]:
# Plot settings
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

# **Funciones**

# **Código Principal**

Para ingresar el bounding box puede usarse el siguiente [enlace](https://norbertrenner.de/osm/bbox.html).

In [ ]:
# Nombre del conjunto de datos
dataset = "reanalysis-era5-single-levels-monthly-means"

# Diseño de la solicitud al servidor
request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": [
        "2m_dewpoint_temperature",
        # "2m_temperature",
        # "surface_pressure",
        # "10m_wind_speed",
        # "evaporation",
        # "runoff"
    ],
    "year": [
        "2014", "2015", "2016",
        "2017", "2018", "2019",
        "2020", "2021", "2022",
        "2023", "2024"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "time": ["00:00"],
    "data_format": "netcdf",
    "download_format": "unarchived",
    "area": [-30.05, -58.55, -34.96, -52.99]
}

# Nombre del archivo. Se puede guardar en una carpeta de Drive agregando la debida ruta
target = 'download.nc'


# Envío de la solicitud y descarga de los datos.
client = cdsapi.Client(key=apikey, url=url)
client.retrieve(dataset, request, target)

2024-12-17 01:34:50,871 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
INFO:datapi.legacy_api_client:[2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-12-17 01:34:50,878 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.le

31f51d1163d25d82b6c577e113f2134b.nc:   0%|          | 0.00/150k [00:00<?, ?B/s]

'download.nc'

In [ ]:
# Una vez descargados los datos se abren en mediante xarray
ds = xr.open_dataset('/content/download.nc')

# Se les fija el CRS (los datos de ERA5 están en el WGS84)
ds.rio.write_crs("epsg:4326", inplace=True)

ds

<xarray.Dataset> Size: 245kB
Dimensions:      (valid_time: 131, latitude: 20, longitude: 23)
Coordinates:
    number       int64 8B ...
  * valid_time   (valid_time) datetime64[ns] 1kB 2014-01-01 ... 2024-11-01
  * latitude     (latitude) float64 160B -30.21 -30.46 -30.71 ... -34.71 -34.96
  * longitude    (longitude) float64 184B -58.55 -58.3 -58.05 ... -53.3 -53.05
    expver       (valid_time) <U4 2kB ...
    spatial_ref  int64 8B 0
Data variables:
    d2m          (valid_time, latitude, longitude) float32 241kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [ ]:
# Se puede ver toda la información que contienen los datos de ERA5
# descargados
ds['d2m'].attrs

{'GRIB_paramId': 168,
 'GRIB_dataType': 'an',
 'GRIB_numberOfPoints': 460,
 'GRIB_typeOfLevel': 'surface',
 'GRIB_stepUnits': 1,
 'GRIB_stepType': 'avgua',
 'GRIB_gridType': 'regular_ll',
 'GRIB_uvRelativeToGrid': 0,
 'GRIB_NV': 0,
 'GRIB_Nx': 23,
 'GRIB_Ny': 20,
 'GRIB_cfName': 'unknown',
 'GRIB_cfVarName': 'd2m',
 'GRIB_gridDefinitionDescription': 'Latitude/Longitude Grid',
 'GRIB_iDirectionIncrementInDegrees': 0.25,
 'GRIB_iScansNegatively': 0,
 'GRIB_jDirectionIncrementInDegrees': 0.25,
 'GRIB_jPointsAreConsecutive': 0,
 'GRIB_jScansPositively': 0,
 'GRIB_latitudeOfFirstGridPointInDegrees': -30.21,
 'GRIB_latitudeOfLastGridPointInDegrees': -34.96,
 'GRIB_longitudeOfFirstGridPointInDegrees': -58.55,
 'GRIB_longitudeOfLastGridPointInDegrees': -53.05,
 'GRIB_missingValue': 3.4028234663852886e+38,
 'GRIB_name': '2 metre dewpoint temperature',
 'GRIB_shortName': '2d',
 'GRIB_totalNumber': 0,
 'GRIB_units': 'K',
 'long_name': '2 metre dewpoint temperature',
 'units': 'K',
 'standard_name

In [ ]:
# Se carga el shape de Uruguay
departments_gdf = gpd.read_file('/ruta/shp/departamentos/en/este/caso/es/uruguay')

In [ ]:
# Ancho de la columna del documento en pulgadas
ancho_columna = 6.5314961

# Relación de aspecto
aspect_ratio = (4/3)/2

# Altura de la figura
altura_figura = ancho_columna/aspect_ratio

# Contador
cont = 0

# Crear figura y ejes
# fig, axes = plt.subplots(4,3, sharex = True, sharey = True)
# fig, axes = plt.subplots(4,3,figsize=(ancho_columna, altura_figura), sharex = True, sharey = True)
fig, axes = plt.subplots(4,3,figsize=(8.3, 8.7), sharex = True, sharey = True)
fig.suptitle(ds['d2m'].attrs['long_name'])

# Se grafican los datos de la variable para el año 2014
for ax in axes.flat:

    grafica = ds['d2m'][cont,:,:].plot(cmap=cmap, ax=ax, add_colorbar = False)
    departments_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=0.25)
    ax.set_title(ds['d2m']['valid_time'][cont].dt.strftime('%B %Y').values)
    ax.set_ylabel("")
    ax.set_xlabel("")
    # ax.set_aspect('equal')


    # Formateo de la barra de color
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(grafica, orientation='vertical', cax=cax)
    cbar.set_label('°K')

    cont += 1

fig.supxlabel("Longitude")
fig.supylabel("Latitude")
fig.tight_layout()
# plt.subplots_adjust(left=0.0, bottom=0.0, right=1.5, top=1.7, wspace=0.3, hspace=0.3); plt.show()
plt.show()